# Validation

# 1. Introduction : pourquoi valider ?

Parce qu'on a des choix à faire ! Il faut choisir, déjà, la ou les classes de modèles. Il y en a un grand nombre : $k$ plus proches voisins, arbres de décision, forêt aléatoire, régressions linéaires, régressions logistiques, "support vector machines" (SVM),  réseaux de neurones...

Il faut aussi choisir les _hyperparamètres_ des modèles. Un hyperparamètre est un paramètre qui n'évolue pas au cours de l'entraînement, par exemple : le nombre de voisins considérés par un modèle de $k$ plus proches voisins (par défaut $5$ dans la librairie `scikit-learn`).

Avoir une vague idée du fonctionnement des différents modèles et du rôle joué par les différents hyperparamètres peut nous aider à faire ces choix. Cependant, le mieux reste de mesurer objectivement les performances. C'est précisément le rôle de la validation : mesurer les performances de différentes classes de modèles et de différents hyperparamètres afin de sélectionner la meilleure option.



# 2. Valider avec un jeu de validation

Pour ce premier exercice pratique, nous allons nous concentrer sur un nouveau problème : celui de prédire le taux de victoire d'un pokemon dans un duel. On prédit un pourcentage, c'est-à-dire un nombre, on a donc affaire à un problème de **régression**.

Importez les jeux de données `pokedex.csv` et `combats.csv`. Étudiez leur structure en affichant les dix premières lignes de chaque jeu de données.

In [1]:
import pandas as pd

dfPokedex = pd.read_csv('pokedex.csv', sep=";")
dfCombats = pd.read_csv('combats.csv')


print("Pokedex:")
print(dfPokedex.iloc[:10,:])
# print("\nCombats:")
# print(dfCombats.head(10))



Pokedex:
   NUMERO               NOM TYPE_1  TYPE_2  POINTS_DE_VIE  POINTS_ATTAQUE  \
0       1        Bulbizarre  Herbe  Poison             45              49   
1       2        Herbizarre  Herbe  Poison             60              62   
2       3        Florizarre  Herbe  Poison             80              82   
3       4   Mega Florizarre  Herbe  Poison             80             100   
4       5         Salamèche    Feu     NaN             39              52   
5       6         Reptincel    Feu     NaN             58              64   
6       7         Dracaufeu    Feu     Vol             78              84   
7       8  Mega Dracaufeu X    Feu  Dragon             78             130   
8       9  Mega Dracaufeu Y    Feu     Vol             78             104   
9      10          Carapuce    Eau     NaN             44              48   

   POINTS_DEFFENCE  POINTS_ATTAQUE_SPECIALE  POINT_DEFENSE_SPECIALE  \
0               49                       65                      65   
1

### Manque-t-il des données dans la pokedex ? 

In [2]:
missing_values = dfPokedex.isnull().sum()
print(missing_values)

NUMERO                       0
NOM                          1
TYPE_1                       0
TYPE_2                     387
POINTS_DE_VIE                0
POINTS_ATTAQUE               0
POINTS_DEFFENCE              0
POINTS_ATTAQUE_SPECIALE      0
POINT_DEFENSE_SPECIALE       0
POINTS_VITESSE               0
NOMBRE_GENERATIONS           0
LEGENDAIRE                   0
dtype: int64


### Si oui, comment traiter ces données manquantes ?

In [3]:
# Sois enlevé completement les lignes avec une valeur null
# dfPokedex = dfPokedex.dropna()

# Sois Remplir les valeurs manquantes

# Pour la colonne NOM
# Option 1: Supprimer la ligne avec le nom manquant
# dfPokedex = dfPokedex.dropna(subset=['NOM'])

# Option 2 (si vous trouvez le nom manquant) : Remplacer manuellement
dfPokedex.loc[dfPokedex['NOM'].isnull(), 'NOM'] = 'Machoc'

# Remplir les valeurs manquantes pour la colonne TYPE_2
dfPokedex['TYPE_2'] = dfPokedex['TYPE_2'].fillna('Aucun')




### Ajoutez trois nouvelles colonnes au dataframe `pokedex` à partir des données dans `combats` : le nombre de combats menés, le nombre de combats gagnés, et le pourcentage de combats gagnés. Attention, certains pokémons n'ont jamais combattu...

In [276]:

nbCombats = dfCombats.value_counts('First_pokemon') + dfCombats.value_counts("Second_pokemon")
nbCombats.index -= 1
dfPokedex["NB_COMBATS"] = nbCombats
dfPokedex['NB_COMBATS'].fillna(0, inplace=True)


nbCombatsGagner = dfCombats.value_counts("Winner")
nbCombatsGagner.index -= 1
dfPokedex["NB_COMBATS_GAGNER"] = nbCombatsGagner
dfPokedex["NB_COMBATS_GAGNER"].fillna(0, inplace=True)



pourcentageCombatsGagner = (nbCombatsGagner / nbCombats)
dfPokedex["POURCENTAGE_COMBATS_GAGNER"] = pourcentageCombatsGagner
dfPokedex = dfPokedex[dfPokedex["POURCENTAGE_COMBATS_GAGNER"].notna()]




### Quel est le Pokémon qui a gagné le plus de combats ?

In [277]:
dfPokedex["NB_COMBATS_GAGNER"].sort_values(ascending=False)
dfPokedex.loc[dfPokedex["NB_COMBATS_GAGNER"].idxmax()]

NUMERO                             163
NOM                             Mewtwo
TYPE_1                             Psy
TYPE_2                           Aucun
POINTS_DE_VIE                      106
POINTS_ATTAQUE                     110
POINTS_DEFFENCE                     90
POINTS_ATTAQUE_SPECIALE            154
POINT_DEFENSE_SPECIALE              90
POINTS_VITESSE                     130
NOMBRE_GENERATIONS                   1
LEGENDAIRE                        VRAI
NB_COMBATS                       164.0
NB_COMBATS_GAGNER                152.0
POURCENTAGE_COMBATS_GAGNER    0.926829
Name: 162, dtype: object

### Existe-t-il un pokémon qui n'a jamais gagné en ayant combattu au moin une fois ? 

In [278]:
# Trouve les poke qui on NB_COMBATS_GAGNER = 0 et NB_COMBATS supérieur a 0
dfPokedex[(dfPokedex["NB_COMBATS_GAGNER"] == 0 ) & (dfPokedex["NB_COMBATS"] > 0)]

,NUMERO,NOM,TYPE_1,TYPE_2,POINTS_DE_VIE,POINTS_ATTAQUE,POINTS_DEFFENCE,POINTS_ATTAQUE_SPECIALE,POINT_DEFENSE_SPECIALE,POINTS_VITESSE,NOMBRE_GENERATIONS,LEGENDAIRE,NB_COMBATS,NB_COMBATS_GAGNER,POURCENTAGE_COMBATS_GAGNER


### Affichez les corrélations entre les différentes variables. Quelles variables semblent utiles pour prédire le pourcentage de victoire ?

In [279]:
dfPokedex.corr(numeric_only=True)

,NUMERO,POINTS_DE_VIE,POINTS_ATTAQUE,POINTS_DEFFENCE,POINTS_ATTAQUE_SPECIALE,POINT_DEFENSE_SPECIALE,POINTS_VITESSE,NOMBRE_GENERATIONS,NB_COMBATS,NB_COMBATS_GAGNER,POURCENTAGE_COMBATS_GAGNER
NUMERO,1.000000,0.101949,0.103613,0.101527,0.088006,0.088499,0.007267,0.983272,-0.039652,0.050732,0.059498
POINTS_DE_VIE,0.101949,1.000000,0.417427,0.265230,0.363244,0.409110,0.179423,0.060534,0.004109,0.259881,0.258006
POINTS_ATTAQUE,0.103613,0.417427,1.000000,0.464539,0.395211,0.288078,0.382310,0.050805,0.056463,0.502255,0.500181
POINTS_DEFFENCE,0.101527,0.265230,0.464539,1.000000,0.237592,0.490118,0.025762,0.048046,0.104530,0.142628,0.129426
POINTS_ATTAQUE_SPECIALE,0.088006,0.363244,0.395211,0.237592,1.000000,0.529276,0.470548,0.034729,0.008521,0.474116,0.478940
POINT_DEFENSE_SPECIALE,0.088499,0.409110,0.288078,0.490118,0.529276,1.000000,0.276715,0.030303,0.034757,0.328222,0.324218
POINTS_VITESSE,0.007267,0.179423,0.382310,0.025762,0.470548,0.276715,1.000000,-0.028512,-0.042596,0.918536,0.937742
NOMBRE_GENERATIONS,0.983272,0.060534,0.050805,0.048046,0.034729,0.030303,-0.028512,1.000000,-0.042744,0.011232,0.021021
NB_COMBATS,-0.039652,0.004109,0.056463,0.104530,0.008521,0.034757,-0.042596,-0.042744,1.000000,0.136214,-0.037894
NB_COMBATS_GAGNER,0.050732,0.259881,0.502255,0.142628,0.474116,0.328222,0.918536,0.011232,0.136214,1.000000,0.980870


## 2. a) Ne pas confondre validation et évaluation !

On est souvent tenté de valider avec le jeu d'évaluation, mais cela enlève tout l'intérêt de préparer un jeu d'évaluation séparé du jeu d'entraînement. On va donc créer un jeu de validation spécialement pour l'occasion.

Séparez les données de la `pokedex` en trois dataframes : `pokedex_train` (60% des observations), `pokedex_validation` (20% des observations) et `pokedex_test` (20 % des observations). Vous pouvez le faire à la main, mais vous pouvez aussi utiliser `train_test_split` de la librairie `scikit-learn` ;)

In [280]:
from sklearn.model_selection import train_test_split

pokedex_train, temp = train_test_split(dfPokedex, test_size=0.4, random_state=42)
pokedex_validation, pokedex_test = train_test_split(temp, test_size=0.5, random_state=42)


Séparez maintenant vos trois dataframes en `X` et `Y`. Dans notre cas, les variables prédictives sont :
- les points de vie
- le niveau d'attaque
- le niveau de défense
- le niveau d'attaque spéciale
- le niveau de défense spéciale
- la vitesse et la génération du Pokémon

Et la variable cible (la prédiction) est :
- le pourcentage de victoire

In [281]:
# Définir les colonnes pour les features et la cible
features_columns = ['POINTS_DE_VIE', 'POINTS_ATTAQUE', 'POINTS_DEFFENCE', 'POINTS_ATTAQUE_SPECIALE', 'POINT_DEFENSE_SPECIALE', 'POINTS_VITESSE', 'NOMBRE_GENERATIONS']
target_column = 'POURCENTAGE_COMBATS_GAGNER'


# Séparation des données pour l'ensemble d'entraînement
X_train = pokedex_train[features_columns]
Y_train = pokedex_train[target_column]

# Séparation des données pour l'ensemble de validation
X_validation = pokedex_validation[features_columns]
Y_validation = pokedex_validation[target_column]

# Séparation des données pour l'ensemble de test
X_test = pokedex_test[features_columns]
Y_test = pokedex_test[target_column]

## 2. b) Différentes classes de modèles

Avec `scikit-learn` entraînez les modèles suivants sur les données d'entraînement et évaluez-les avec les données de validation.

Un arbre de décision (`DecisionTreeRegressor`) :

In [282]:
from sklearn.tree import DecisionTreeRegressor

modelDT = DecisionTreeRegressor()

# Entraînement du modèle
modelDT.fit(X_train, Y_train)

# Prédiction sur l'ensemble de validation
modelDT.score(X_validation, Y_validation)


0.9094506128318153

Une forêt aléatoire (`RandomForestRegressor`) :

In [283]:
from sklearn.ensemble import RandomForestRegressor

modelRfr = RandomForestRegressor()

# Entraînement du modèle
modelRfr.fit(X_train, Y_train)

# Prédiction sur l'ensemble de validation
modelRfr.score(X_validation, Y_validation)


0.9585296361519205

Une régression linéaire (`LinearRegression`) :

In [284]:
from sklearn.linear_model import LinearRegression

modelLr = LinearRegression()

# Entraînement du modèle
modelLr.fit(X_train, Y_train)

# Prédiction sur l'ensemble de validation
modelLr.score(X_validation, Y_validation)


0.9201557515639778

Une régression "Lasso" (`Lasso`) :

In [285]:
from sklearn.linear_model import Lasso

modelLasso = Lasso()

# Entraînement du modèle
modelLasso.fit(X_train, Y_train)

# Prédiction sur l'ensemble de validation
modelLasso.score(X_validation, Y_validation)


0.890024214101365

Lequel de ces modèles semble le plus performant ?

_À compléter_

# 3 La validation croisée

Créer un jeu de validation, c'est bien, mais ça limite la quantité de données qu'on peut utiliser pour l'entraînement. Une alternative est de faire de la validation croisée.

## 3. a) Choisir un hyperparamètre avec une "Grid Search"

La classe de modèle `Lasso` accepte plusieurs hyperparamètres, parmi eux `alpha`. Sans rentrer dans les détails de ce que signifient ces hyperparamètres, on souhaiterait choisir la meilleure valeur possible. On va donc utiliser une méthode de recherche exhaustive, aussi appelée **grid search**.

Utilisez la méthode `GridSearchCV` de `scikit-learn` pour trouver la meilleure valeur pour `alpha`.

In [287]:
from sklearn.model_selection import GridSearchCV
import numpy as np

param_grid = {"alpha": np.linspace(0.5, 10, 10)}

lasso = Lasso()
gsc = GridSearchCV(lasso, param_grid, cv=5)
gsc.fit(X_train, Y_train)
gsc.best_params_

{'alpha': 0.5}

In [ ]:
lasso2 = Lasso()
lasso2.fit(X_train, Y_train)
lasso2.score(X_validation, Y_validation)
